In [45]:
import numpy as np
from IPython.core.debugger import set_trace
import re
import difflib
import pandas as pd
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import requests
import time
import random
import pickle
from lxml.html import fromstring
from collections import Counter
from requests.packages.urllib3.util import Retry
from requests.adapters import HTTPAdapter
from requests import Session, exceptions
import sys
import pymongo
from pymongo import MongoClient
from newspaper import Article
import json
from string import digits
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
from nltk.stem.porter import *
from collections import Counter 
from nltk.corpus import wordnet # To get words in dictionary with their parts of speech
from nltk.stem import WordNetLemmatizer # lemmatizes word based on it's parts of speech
from nltk.corpus import stopwords 
from textblob import TextBlob
import spacy
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from nltk.corpus import cmudict
from sklearn.feature_extraction.text import TfidfVectorizer
import gensim
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from langdetect import detect
from numpy import dot
from numpy.linalg import norm
import rake_nltk
sys.path.append("/Users/rohanramesh/Documents/GitHub/Insight_writers/Dash_to_server/")
from text_processing import ProcessArticle as pa
import suggestions as s
from sklearn.metrics.pairwise import cosine_similarity
import scipy

In [2]:
# data load from muckrack
with open('/Users/rohanramesh/Documents/Insight/data_bball_writers/Attempt3_mr_scrape.pickle', 'rb') as handle:
    scrapevar = pickle.load(handle)
    
# load writer df
with open('/Users/rohanramesh/Documents/Insight/data_bball_writers/writer_df.pickle', 'rb') as handle:
    writer_df = pickle.load(handle)

In [3]:
# Establish connection with Mongo database
client = MongoClient()
mydb = client["testinsightdb"]

In [172]:
# to get the number of words per article
nwords_article = []
for curr_name in writer_df['website_name']:
    # point to right collection in database
    currcol = mydb[curr_name]
    print(curr_name)
    # get all articles
    y = currcol.find({"name": curr_name})
    # iterate through all articles for that author
    idx = 1
    for curr_art in y:
        # tokenize words and append to get number of words per article
        a = word_tokenize(clean_text(curr_art['article']))
        nwords_article.append(len(a))

zachlowe_nba
adrian-wojnarowski
lee-jenkins
howardbeck
marc-stein
ethan-sherwood-strauss
jason-concepcion
kevin-arnovitz
tom-haberstroh
nate-duncan
zach-harper
brian-windhorst
sam_amick
david-aldridge
ramona-shelburne
jpdabrams
kevin-pelton
marc-j-spears
matt-moore
shams-charania
kevin-p-oconnor
chris-haynes
dave-mcmenamin
shea-serrano
ian-begley
rachel-nichols
ben-golliver
mike-prada
robmahoney
tim-macmahon
chris-herring
billsimmons
jonathan-tjarks
amin-elhassen
bobby-marks
danny-leroux
chris-mannix
dan-devine
michael-pina
thompsonscribe
ben-golliver
ben-falk
ian-levy
tim-bontemps
henry-abbott
scott-rafferty
ja-dubin
michael-lee
alex-kennedy
derek-bodner
tom-ziller
chris-ballard
josh-eberley
adi-joseph
adam-mares
sam-vecenie
meet-katie-nolan
andrew-sharp
david-thorpe
royce-webb
ericpincus
nick-sciria
paul-flannery
timkawakami
keith-smith
joshrobbins
jeff-zillgitt
jon-krawczynski
seerat-sohi
israel-gutierrez
ben-rohrbach
mika-honkasalo
ian-oconnor-1027184
kevin-ferrigan
chris-vernon
tr

# Tf-IDF
Try Tf-IDF as approach for suggesting content. Will also try word2vec for lower dimensional reprensentation. Looks at ratio of word frequency to corpus frequency

In [84]:
# gather and preprocess data for training tf-idf model
data_into_tfidf = []
# iterate over each name in the database
for curr_name in writer_df['website_name']:
    # current collection
    currcol = mydb[curr_name]
    print(curr_name)
    # all articles in the collection
    y = currcol.find({"name": curr_name})
    # iterate through all articles for that author
    idx = 1
    for curr_art in y:
        # do preprocessing of text using process articles class
        curr = pa(curr_art['article'])
        curr.clean_text()
        data_into_tfidf.append(clean_text(curr.cleanedtext))
        # don't inlude more than 250 articles from any one author
        idx += 1
        if idx > 250:
            print('breaking')
            break

zachlowe_nba
breaking
adrian-wojnarowski
breaking
lee-jenkins
howardbeck
marc-stein
ethan-sherwood-strauss
jason-concepcion
kevin-arnovitz
tom-haberstroh
nate-duncan
zach-harper
brian-windhorst
sam_amick
david-aldridge
ramona-shelburne
jpdabrams
kevin-pelton
marc-j-spears
matt-moore
shams-charania
kevin-p-oconnor
chris-haynes
dave-mcmenamin
shea-serrano
ian-begley
rachel-nichols
ben-golliver
mike-prada
robmahoney
tim-macmahon
chris-herring
billsimmons
jonathan-tjarks
amin-elhassen
bobby-marks
danny-leroux
chris-mannix
dan-devine
michael-pina
thompsonscribe
ben-golliver
ben-falk
ian-levy
tim-bontemps
henry-abbott
scott-rafferty
ja-dubin
michael-lee
alex-kennedy
derek-bodner
tom-ziller
chris-ballard
josh-eberley
adi-joseph
adam-mares
sam-vecenie
meet-katie-nolan
andrew-sharp
david-thorpe
royce-webb
ericpincus
nick-sciria
paul-flannery
timkawakami
keith-smith
joshrobbins
jeff-zillgitt
jon-krawczynski
seerat-sohi
israel-gutierrez
ben-rohrbach
mika-honkasalo
ian-oconnor-1027184
kevin-ferrig

In [12]:
# train tf-idf vectorizer
vec = TfidfVectorizer()
vec.fit(data_into_tfidf)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [13]:
# create vector of each article and put in list
output_vec_tf = []
for curr_art in data_into_tfidf:
    output_vec_tf.append(vec.transform([curr_art]))


# Word2Vec
Try Word2Vec as an approach as well for each article. Shallow neural network that will look at context of word to predict the actual word (using continuous bag of words approach for now). Will represent each word as a n dimensional vector (lower dimension than tf-idf) and will average across vectors to get "content" of article.

In [11]:
# build corpus for training word2vec model
# saving three values - list containing preprocessed text for each article, 
# url information for each article
article_string = []
url_string = []
# data_into_tfidf = [] # if saving for tf-idf
# iterate through each name
for curr_name in writer_df['website_name']:
    # identify appropriate collection
    currcol = mydb[curr_name]
    print(curr_name)
    # get all articles for that writer
    y = currcol.find({"name": curr_name})
    for curr_article in y:
        # check if article is english and if not then continue
        a = s.detect_nonenglish(curr_article['article'])
        if a == True:
            continue
        # for word2vec
        lem_art = s.lemstr(gensim.utils.simple_preprocess(curr_article['article']))
        article_string.append(lem_art)
        # if building for tf-idf
#         data_into_tfidf.append(curr.cleanedtext)
        url_string.append(curr_article['article_url'])



zachlowe_nba
adrian-wojnarowski
lee-jenkins
howardbeck
marc-stein
ethan-sherwood-strauss
jason-concepcion
kevin-arnovitz
tom-haberstroh
nate-duncan
zach-harper
brian-windhorst
sam_amick
david-aldridge
ramona-shelburne
jpdabrams
kevin-pelton
marc-j-spears
matt-moore
shams-charania
kevin-p-oconnor
chris-haynes
dave-mcmenamin
shea-serrano
ian-begley
rachel-nichols
ben-golliver
mike-prada
robmahoney
tim-macmahon
chris-herring
billsimmons
jonathan-tjarks
amin-elhassen
bobby-marks
danny-leroux
chris-mannix
dan-devine
michael-pina
thompsonscribe
ben-golliver
ben-falk
ian-levy
tim-bontemps
henry-abbott
scott-rafferty
ja-dubin
michael-lee
alex-kennedy
derek-bodner
tom-ziller
chris-ballard
josh-eberley
adi-joseph
adam-mares
sam-vecenie
meet-katie-nolan
andrew-sharp
david-thorpe
royce-webb
ericpincus
nick-sciria
paul-flannery
timkawakami
keith-smith
joshrobbins
jeff-zillgitt
jon-krawczynski
seerat-sohi
israel-gutierrez
ben-rohrbach
mika-honkasalo
ian-oconnor-1027184
kevin-ferrigan
chris-vernon
tr

In [158]:
# try word2vec model
# vector length for each word
vec_size = 150
model = gensim.models.Word2Vec(
        article_string,
        size=vec_size,
        window=12, # total number of words to account for
        min_count=2,
        workers=8)
model.train(article_string, total_examples=len(article_string), epochs=10)


(112255268, 146738100)

In [13]:
# get keyed vectors saved or loaded for faster calculations
# model.save("/Users/rohanramesh/Documents/Insight/data_bball_writers/word2vec_model.model")
# model.wv.save("/Users/rohanramesh/Documents/Insight/data_bball_writers/word2vec_model_kv.kv")
kv = gensim.models.KeyedVectors.load(
    "/Users/rohanramesh/Documents/Insight/data_bball_writers/word2vec_model_kv.kv", mmap='r')

In [165]:
# for each article lets build a vector that corresponds to the entire article by taking the mean
# across all words included in the article
article_mat = np.ndarray(shape=[np.shape(article_string)[0],vec_size])
# iterate over each article
for idx in range(0,len(article_string)):
    # check if words are in vocab and if not then don't include those words
    a = [r for r in article_string[idx] if r in model.wv.vocab]
    # build matrix for each word where each row is a word and columns = word embedding vector
    array_for_art = np.ndarray(shape=[len(a),vec_size])
    for i in range(0,len(a)):
        array_for_art[i,:] = model.wv.get_vector(a[i])
    # take mean across all words to get the vector for the article
    article_mat[idx,:] = np.mean(array_for_art, axis=0)

# build similarity matrix across all articles using cosine similarity
all_art_cos_sim = np.ndarray(shape=[len(article_mat),len(article_mat)])
for i in range(0,len(article_mat)):
    for j in range(0,len(article_mat)):
        # calculate cos sim between all articles
        all_art_cos_sim[i,j] = s.cos_sim(article_mat[i,:],article_mat[j,:])

In [172]:
# build dataframe with word2vec dim reduced vectors
w2v_df = pd.DataFrame()
for i in range(0,len(article_mat)):
    w2v_df[url_string[i]] = article_mat[i,:]

In [210]:
# save w2v as pickle for loading
w2v_df.to_pickle('/Users/rohanramesh/Documents/Insight/data_bball_writers/word2vec_trained.pickle')

# Doc2Vec
Try Doc2Vec as an approach as well for each article. Apparently tuning takes a bit more work with doc2vec so might need to sink more time to get this to work as well as word2vec. Have to reformat input to model to accomade for "tagged" documents

In [99]:
# trying Doc2vec - have to load in data plus tags
docs = []
# iterate through each writer and collection to get each article
for curr_name in writer_df['website_name']:
    currcol = mydb[curr_name]
    print(curr_name)
    y = currcol.find({"name": curr_name})
    for curr_article in y:
        # check if english then lemmatize and append for doc structure in doc2vec
        a = s.detect_nonenglish(curr_article['article'])
        if a == True:
            continue
        lem_art = s.lemstr(gensim.utils.simple_preprocess(curr_article['article']))
        docs.append(gensim.models.doc2vec.TaggedDocument(lem_art, curr_article['article_url']))


In [11]:
# put into proper doc2vec formatting using their TaggedDocument approach
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(common_texts)]


In [25]:
# build doc2vec model and vocabulary
model = gensim.models.Doc2Vec(size=200, window=10, min_count=2, 
                              workers=4,alpha=0.025, min_alpha=0.025) # use fixed learning rate
model.build_vocab(docs)


/anaconda3/envs/insight_new/lib/python3.6/site-packages/gensim/models/doc2vec.py:366: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


In [30]:
# train model with decreasing learning rate
for epoch in range(10):
    model.train(docs, total_examples=model.corpus_count, epochs=1)
    model.alpha -= 0.002 # decrease the learning rate
    model.min_alpha = model.alpha # fix the learning rate, no deca
    model.train(docs, total_examples=model.corpus_count, epochs=1)

In [64]:
# get the vector representations of each article using Doc2vec using same approach as for word2vec
article_mat = np.ndarray(shape=[np.shape(docs)[0], model.vector_size])
for idx in range(0,np.shape(docs)[0]):
    # make sure in vocab
    a = [r for r in docs[idx][0] if r in model.wv.vocab]
    # average across all words
    array_for_art = np.ndarray(shape=[len(a), model.vector_size])
    for i in range(0,len(a)):
        array_for_art[i,:] = model.wv.get_vector(a[i])
    article_mat[idx,:] = np.mean(array_for_art, axis=0)

In [67]:
# build dataframe with doc2vec dim reduced vectors and save
d2v_df = pd.DataFrame()
for i in range(0,len(article_mat)):
    d2v_df[docs[i][1]] = article_mat[i,:]
    
# save doc2vec model and keyedvectors
model.save("/Users/rohanramesh/Documents/Insight/data_bball_writers/doc2vec_model.model")
model.wv.save("/Users/rohanramesh/Documents/Insight/data_bball_writers/doc2vec_model_kv.kv")
d2v_df.to_pickle("/Users/rohanramesh/Documents/Insight/data_bball_writers/d2v_df.pickle")